<a href="https://colab.research.google.com/github/linyuehzzz/traffic_data_aggregation/blob/master/data_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount on Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add file path**

In [6]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/traffic_data_aggregation")
!ls

data  data_agg.ipynb


**Prepare TomTom data**

In [5]:
%cd data
import csv

def get_response(url):
  req = urlopen(url)
  res = req.read().decode()
  temp = json.loads(res)
  return temp


def get_key(file):
  with open(file, "r") as f:
    return f.read()


def get_tomtom_all(file, key):
  with open(file, "r") as f:
    all_lines = csv.DictReader(f)
    for line in all_lines:
      lat = line['lat']
      lon = line['lon']
      url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/xml?point=" + \
      lat + "%2C" + lon + "&unit=KMPH&key=" + key

      speed = get_response(url)["currentSpeed"]
      speed_ff = get_response(url)["freeFlowSpeed"] # free flow speed
      travel_time = get_response(url)["currentTravelTime"]
      travel_time_ff = get_response(url)["currentTravelTime"]


file_key = "key.txt"
key = get_key(file_key)
print(key)


[Errno 2] No such file or directory: 'data'
/content/gdrive/My Drive/Colab Notebooks/traffic_data_aggregation/data
Tt3shrV6AERrnZz1ejpaYnXKOCsorPVk
